## Satellite data processing guide
The document explains how to access gridded satellite data in netCDF format, arichived in OPeNDAP. Popular sources of satellite data are NASA, NOAA, and Reanalysis data from ECMWF.  

## Import required packages

In [1]:
import xarray as xr
import requests

## Access NOAA data through url and filter to a given time and location
First we need to find the NOAA datasets that are archived in OpeNDAP. A comprehensive list of NOVAA gridded products which can be found in OpeNDAP is available at https://psl.noaa.gov/data/gridded/. After we select a particular product then check it in OpeNDAP and copy its url. After then open url with xarray and then check the metadata inside it, as demonstrated below. Then this parameter will be used in successive stage to filter and acquire data. 



In [2]:
url = 'http://psl.noaa.gov/thredds/dodsC/Datasets/cpc_global_precip/precip.day.1991-2020.ltm.nc'
noaa_pr=xr.open_dataset(url)

#print data to check metadata
noaa_pr

/Users/farid/Abaconda3/anaconda3/envs/geo_env/lib/python3.10/site-packages/xarray/coding/times.py:688: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/Users/farid/Abaconda3/anaconda3/envs/geo_env/lib/python3.10/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)


<xarray.Dataset>
Dimensions:             (lat: 360, lon: 720, time: 365, nbnds: 2)
Coordinates:
  * lat                 (lat) float32 89.75 89.25 88.75 ... -88.75 -89.25 -89.75
  * lon                 (lon) float32 0.25 0.75 1.25 1.75 ... 358.8 359.2 359.8
  * time                (time) object 0001-01-01 00:00:00 ... 0001-12-31 00:0...
Dimensions without coordinates: nbnds
Data variables:
    climatology_bounds  (time, nbnds) datetime64[ns] ...
    precip              (time, lat, lon) float32 ...
    valid_yr_count      (time, lat, lon) float32 ...
Attributes:
    Conventions:                    CF-1.0
    version:                        V1.0
    history:                        Created 2021/02/09 by doDayLTM
    title:                          CPC GLOBAL PRCP V1.0
    dataset_title:                  CPC GLOBAL PRCP V1.0
    Source:                         ftp://ftp.cpc.ncep.noaa.gov/precip/CPC_UN...
    References:                     https://www.psl.noaa.gov/data/gridded/dat...
    not_missing_threshold_percent:  minimum 3% values input to have non-missi...

In [4]:
x=71 #define longitude to take data
y=33 #define latitue to take data
var = 'precip' # check metadata, as was shown in above, desired variable is precip (precipitation)

pr = noaa_pr.precip.sel(lon=x,lat=y,method='nearest')
pr = pr.to_dataframe()

#export data to a csv table and it should be as below.
#pr.to_csv('directory to save data.csv')
pr

,lat,lon,precip
time,,,
0001-01-01 00:00:00,33.25,71.25,0.506324
0001-01-02 00:00:00,33.25,71.25,0.171190
0001-01-03 00:00:00,33.25,71.25,0.711516
0001-01-04 00:00:00,33.25,71.25,0.831335
0001-01-05 00:00:00,33.25,71.25,0.827286
...,...,...,...
0001-12-27 00:00:00,33.25,71.25,0.520333
0001-12-28 00:00:00,33.25,71.25,0.369567
0001-12-29 00:00:00,33.25,71.25,0.216906


## Access NASA data and filter to a given time and location
All the NASA data that are available in OPenDAP are available at https://podaac-opendap.jpl.nasa.gov/opendap/. All the available in the site can be processed successfully. However, I am not able to access the desirable dataset, that is not listed in the mentioned website but it is available in OpENDAP. Hopfully, I can a find way out of the error to access gpm data. 


In [9]:
url='https://gpm1.gesdisc.eosdis.nasa.gov/opendap/GPM_L3/GPM_3IMERGDE.06/2000/06/3B-DAY-E.MS.MRG.3IMERG.20000601-S000000-E235959.V06.nc4'
url2=''

session = requests.Session()
session.auth = ('faridfarhat', 'Stand@2020@2020@')

store = xr.backends.PydapDataStore.open(url, session=session)
ds = xr.open_dataset(store)

HTTPError: Failed to decode variable 'time_bnds': 302 Found
<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>302 Found</title>
</head><body>
<h1>Found</h1>
<p>The document has moved <a href="https://urs.earthdata.nasa.gov/oauth/authorize/?scope=uid&amp;app_type=401&amp;client_id=e2WVk8Pw6weeLUKZYOxvTQ&amp;response_type=code&amp;redirect_uri=https%3A%2F%2Fgpm1.gesdisc.eosdis.nasa.gov%2Fdata-redirect&amp;state=aHR0cHM6Ly9ncG0xLmdlc2Rpc2MuZW9zZGlzLm5hc2EuZ292L29wZW5kYXAvR1BNX0wzL0dQTV8zSU1FUkdERS4wNi8yMDAwLzA2LzNCLURBWS1FLk1TLk1SRy4zSU1FUkcuMjAwMDA2MDEtUzAwMDAwMC1FMjM1OTU5LlYwNi5uYzQuZG9kcz90aW1lX2JuZHMlNUIwOjE6MCU1RCU1QjA6MTowJTVE">here</a>.</p>
</body></html>

This is redirect error. These should not usually raise an error in pydap beacuse redirects are handled implicitly. If it failed it is likely due to a circular redirect.

In [10]:
from pydap.client import open_url
from pydap.cas.urs import setup_session

url='https://gpm1.gesdisc.eosdis.nasa.gov/opendap/GPM_L3/GPM_3IMERGDE.06/2000/06/3B-DAY-E.MS.MRG.3IMERG.20000601-S000000-E235959.V06.nc4'

session = setup_session('faridfarhat', 'Stand@2020@2020@')
dataset = open_url(url, session=session)

In [18]:
xr.open_dataset(dataset,engine='pydap')

AttributeError: '<class 'pydap.model.DatasetType'>' object has no attribute 'decode'